In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_122123/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 10:01:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/11 10:01:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/11 10:01:50 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
data = [['1', '1', '10', '1', 'completed', '2013-10-01'], ['2', '2', '11', '1', 'cancelled_by_driver', '2013-10-01'], ['3', '3', '12', '6', 'completed', '2013-10-01'], ['4', '4', '13', '6', 'cancelled_by_client', '2013-10-01'], ['5', '1', '10', '1', 'completed', '2013-10-02'], ['6', '2', '11', '6', 'completed', '2013-10-02'], ['7', '3', '12', '6', 'completed', '2013-10-02'], ['8', '2', '12', '12', 'completed', '2013-10-03'], ['9', '3', '10', '12', 'completed', '2013-10-03'], ['10', '4', '13', '12', 'cancelled_by_driver', '2013-10-03']]
trips = pd.DataFrame(data, columns=['id', 'client_id', 'driver_id', 'city_id', 'status', 'request_at']).astype({'id':'Int64', 'client_id':'Int64', 'driver_id':'Int64', 'city_id':'Int64', 'status':'object', 'request_at':'object'})

data = [['1', 'No', 'client'], ['2', 'Yes', 'client'], ['3', 'No', 'client'], ['4', 'No', 'client'], ['10', 'No', 'driver'], ['11', 'No', 'driver'], ['12', 'No', 'driver'], ['13', 'No', 'driver']]
users = pd.DataFrame(data, columns=['users_id', 'banned', 'role']).astype({'users_id':'Int64', 'banned':'object', 'role':'object'})


df_person = spark.createDataFrame(trips)
# Register the DataFrame as a temporary view
df_person.createOrReplaceTempView("Trips")

df_person = spark.createDataFrame(users)
# Register the DataFrame as a temporary view
df_person.createOrReplaceTempView("Users")



In [4]:
spark.sql("select * from Trips").show()
spark.sql("select * from Users").show()

+---+---------+---------+-------+-------------------+----------+
| id|client_id|driver_id|city_id|             status|request_at|
+---+---------+---------+-------+-------------------+----------+
|  1|        1|       10|      1|          completed|2013-10-01|
|  2|        2|       11|      1|cancelled_by_driver|2013-10-01|
|  3|        3|       12|      6|          completed|2013-10-01|
|  4|        4|       13|      6|cancelled_by_client|2013-10-01|
|  5|        1|       10|      1|          completed|2013-10-02|
|  6|        2|       11|      6|          completed|2013-10-02|
|  7|        3|       12|      6|          completed|2013-10-02|
|  8|        2|       12|     12|          completed|2013-10-03|
|  9|        3|       10|     12|          completed|2013-10-03|
| 10|        4|       13|     12|cancelled_by_driver|2013-10-03|
+---+---------+---------+-------+-------------------+----------+

+--------+------+------+
|users_id|banned|  role|
+--------+------+------+
|       1|    

In [47]:
query = """
	select
		t.request_at as Day,
		round(sum(case when t.status like 'cancelled%' then 1 else 0 end) * 1.0 / count(*), 2) as Cancellation_Rate
	from
		Trips t 
  		left join Users uc on t.client_id = uc.users_id
		left join Users ud on t.driver_id = ud.users_id
	where uc.banned <> 'Yes' and ud.banned <> 'Yes'
    and t.request_at between "2013-10-01" and "2013-10-03"
	group by t.request_at
    order by t.request_at asc

"""

In [48]:
spark.sql(query).show()

+----------+-----------------+
|       Day|Cancellation_Rate|
+----------+-----------------+
|2013-10-01|             0.33|
|2013-10-02|             0.00|
|2013-10-03|             0.50|
+----------+-----------------+

